In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import spacy
import re

import os
import numpy as np
import torch

from PIL import Image as PILImage
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from sklearn.model_selection import GroupShuffleSplit,train_test_split
from IPython.display import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/to_upload-2"


In [ ]:
import pandas as pd

train_df = pd.read_csv(f"{BASE_DIR}/train_dataset.csv")
train_df.head()


In [ ]:
train_df.info()

In [ ]:
crowd_df = pd.read_csv(
    f"{BASE_DIR}/CrowdAnnotations.tsv",
    sep="\t",
    header=None
)

crowd_df.columns = [
    "image_name",
    "caption_id",
    "agree_ratio",
    "agree_count",
    "disagree_count"
]

crowd_df.head()


In [ ]:
crowd_df.info()

In [ ]:
expert_df = pd.read_csv(
    f"{BASE_DIR}/ExpertAnnotations.tsv",
    sep="\t",
    header=None
)

expert_df.columns = [
    "image_name",
    "caption_id",
    "expert_1",
    "expert_2",
    "expert_3"
]

expert_df.head()


In [ ]:
expert_df.info()

In [ ]:
test_df = pd.read_csv(
    f"{BASE_DIR}/test_queries.csv",
    sep="|"
)

test_df.head()



  # При загрузке файла `test_queries.csv` возникла ошибка парсинга,для корректной загрузки данных разделитель был задан явно (`sep="|"`)


In [ ]:
test_df.info()

In [ ]:
test_df = test_df.drop(columns=["Unnamed: 0"])
test_df = test_df.rename(columns={"image": "image_name"})


In [ ]:
print("train:", train_df.shape)
print("crowd:", crowd_df.shape)
print("expert:", expert_df.shape)
print("test:", test_df.shape)


## Вывод:

В ходе предварительной работы с датасетом мы проделали следующие шаги:

1. **Загрузка данных:**  
   Все необходимые файлы (`train_dataset.csv`, `CrowdAnnotations.tsv`, `ExpertAnnotations.tsv`, `test_queries.csv`) и папки с изображениями были успешно загружены из Google Drive в среду Colab.

2. **Проверка структуры и типов данных:**  
   - Таблицы `train_df`, `crowd_df` и `expert_df` имеют корректные размеры и типы данных, пропусков не выявлено.  
   - Таблица `test_df` первоначально содержала некорректный разделитель и лишнюю колонку `Unnamed: 0`.

3. **Корректировка данных:**  
   - В `test_df` был явным образом указан разделитель (`sep="|"`) для корректного считывания столбцов.  
   - Удалена лишняя колонка `Unnamed: 0`.  
   - Столбцы приведены к единому формату: `query_id`, `query_text`, `image_name`.

4.На данном этапе датасет полностью подготовлен для следующего этапа PoC — фильтрации запрещённого контента, объединения разметки и формирования обучающей выборки для модели.


1  1. Исследовательский анализ данных

В файле с экспертными мнениями для каждой пары изображение-текст имеются оценки от трёх специалистов. Для решения задачи мы должны эти оценки агрегировать — превратить в одну.

Для каждой пары изображение–текст — 3 оценки экспертов

Шкала от 1 до 4

Нужно получить одну итоговую оценку

В случае полного расхождения (1, 2, 4) — пару можно исключить

Посмотрим распределение экспертных оценок

In [ ]:
expert_df[['expert_1', 'expert_2', 'expert_3']].stack().value_counts().sort_index()

Эксперты чаще всего ставят низкие оценки (1–2)

Полное соответствие (оценка 4) встречается реже всего.

In [ ]:
def aggregate_expert_votes(row):
    votes = [row['expert_1'], row['expert_2'], row['expert_3']] # используем голосование большинства
    counter = Counter(votes)

    most_common = counter.most_common(1)[0]
    if most_common[1] >= 2:
        return most_common[0]
    else:
        return None

In [ ]:
expert_df['expert_score'] = expert_df.apply(aggregate_expert_votes, axis=1)


In [ ]:
expert_df['expert_score'].isna().value_counts()


Всего пар: 5822

Конфликтные оценки : 126

Таким рбразом, доля конфликтов небольшая.

In [ ]:
expert_clean = expert_df.dropna(subset=['expert_score']).copy()


Приведём экспертную оценку к [0, 1]

In [ ]:
expert_clean['expert_score_norm'] = (expert_clean['expert_score'] - 1) / 3


Проведём анализ краудсорсинговых оценок

In [ ]:
expert_df[['expert_1', 'expert_2', 'expert_3']].stack().value_counts().sort_index()


Большинство пар изображение–текст либо:

совсем не соответствуют друг другу,

либо имеют лишь частичное совпадение

Полное или почти полное соответствие (оценки 3–4) встречается существенно реже

In [ ]:
crowd_df['agree_ratio'].value_counts().sort_index()


В большинтсве случаев agree_ratio = 0

Это означает, что ни один исполнитель не подтвердил соответствие

Значения выше 0.5 встречаются редко. Крауд-разметка является очень строгой и подтверждает соответствие только в очевидных ситуациях.

In [ ]:
crowd_df['total_votes'] = (
    crowd_df['agree_count'] + crowd_df['disagree_count']
)

crowd_df['total_votes'].value_counts()



В подавляющем большинстве случаев голосуют 3 человека

Реже — 4–6 исполнителей

In [ ]:
(crowd_df['agree_ratio'] > 0.5).value_counts(normalize=True)


94.3% пар считаются несоответствующими

5.7% — соответствующими.

Крауд-разметка сильно смещена в сторону отрицательного класса и не подходит как единственный источник целевой переменной.

Экспертная разметка — более надёжная.Крауд — более шумный, но массовый

Поэтому используем объединение:

эксперт: 0.6

крауд: 0.4

In [ ]:
expert_ready = expert_clean[[
    'image_name', 'caption_id', 'expert_score_norm'
]]

crowd_ready = crowd_df[[
    'image_name', 'caption_id', 'agree_ratio'
]]


In [ ]:
merged_df = expert_ready.merge(
    crowd_ready,
    on=['image_name', 'caption_id'],
    how='left'
)

merged_df['agree_ratio'] = merged_df['agree_ratio'].fillna(0)


merged_df['target'] = 0.6 * merged_df['expert_score_norm'] + 0.4 * merged_df['agree_ratio']


merged_df.head()

Таким образом,  были изучены экспертные и краудсорсинговые оценки соответствия текста и изображения. Анализ показал, что датасет  несбалансирован: большинство пар изображение–текст не являются релевантными, что подтверждается как экспертной, так и крауд-разметкой.

Экспертные оценки обладают большей информативностью и позволяют различать степень соответствия, тогда как краудсорсинговые оценки носят более строгий и бинарный характер и могут использоваться как дополнительный сигнал уверенности.

На основе агрегированной экспертной оценки и доли согласия крауда была сформирована целевая переменная в диапазоне от 0 до 1.

**Проверка данных**

Согласно законодательству ряда стран, запрещено предоставлять доступ к контенту, содержащему изображения или описания детей младше 16 лет, без согласия законных представителей.

Так как в рамках PoC отсутствует механизм показа дисклеймера, все такие изображения должны быть исключены из обучающей выборки.

In [ ]:
full_df = merged_df.merge(
    train_df,
    left_on='caption_id',
    right_on='query_id',
    how='left'
)


In [ ]:

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [ ]:
BLOCK_LEMMAS = {
    "child", "kid", "boy", "girl",
    "baby", "toddler", "infant"
}


In [ ]:
def contains_blocked_content(text):
    if not isinstance(text, str):
        return False

    text = re.sub(r"[^a-zA-Z]", " ", text).lower()
    doc = nlp(text)

    return any(token.lemma_ in BLOCK_LEMMAS for token in doc)


In [ ]:
full_df["has_children"] = full_df["query_text"].apply(contains_blocked_content)
clean_df = full_df[~full_df["has_children"]].copy()
clean_df.drop(columns=["has_children"], inplace=True)


In [ ]:
full_df['has_children'].value_counts()


In [ ]:
full_df[full_df['has_children']].head(5)


In [ ]:
IMAGES_DIR = "/content/drive/MyDrive/Colab Notebooks/to_upload-2/train_images"


In [ ]:
blocked_samples = full_df[full_df['has_children']]['image_name'].unique()[:5]


In [ ]:
for img_name in blocked_samples:
    img_path = os.path.join(IMAGES_DIR, img_name)
    display(Image(filename=img_path))


Фильтрация выполнена по текстовым описаниям, что может приводить к блокировке изображений без детей, если текст содержит запрещённые слова.

Таким образом, на банном этапе выполнена проверка текстов на наличие запрещённых слов, указывающих на детей.

Тексты были лемматизированы с использованием spaCy, приведены к нижнему регистру и очищены от лишних символов.

Все строки с детьми были помечены и удалены из обучающего датасета.

**Векторизация текстов**

Для векторизации текстовых описаний используем предобученную модель BERT.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

bert_model = bert_model.to(device)
bert_model.eval()


In [ ]:
def text_to_vector(text, max_length=64):
    if not isinstance(text, str):
        return None

    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = bert_model(**inputs)


    vector = outputs.last_hidden_state.mean(dim=1)

    return vector.squeeze().cpu().numpy()


In [ ]:
text_vectors = {}

for caption_id, text in tqdm(
    clean_df[['caption_id', 'query_text']].drop_duplicates().values
):
    vec = text_to_vector(text)
    if vec is not None:
        text_vectors[caption_id] = vec


In [ ]:
len(text_vectors)



In [ ]:
next(iter(text_vectors.values())).shape


Таким образом, текстовые описания были векторизованы с использованием предобученной модели BERT. В результате получены семантические представления фиксированной размерности 768 для 688 уникальных текстовых описаний.

**Векторизация изображений**

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

image_vectors = {}

for img_name in tqdm(clean_df['image_name'].unique()):
    img_path = os.path.join(IMAGES_DIR, img_name)

    if not os.path.exists(img_path):
        continue

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    vec = model.predict(x, verbose=0)[0]
    image_vectors[img_name] = vec


In [ ]:
len(image_vectors)
image_vectors[next(iter(image_vectors))].shape


Векторизация изображений была выполнена с использованием модели ResNet50, предобученной на ImageNet. Для каждого изображения был получен эмбеддинг размерности 2048

**Объединение векторов**

In [ ]:
X = []
y = []
image_names = []
skipped = 0

for _, row in merged_df.iterrows():
    img_name = row['image_name']
    caption_id = row['caption_id']
    target = row['target']


    img_vec = image_vectors.get(img_name)
    txt_vec = text_vectors.get(caption_id)

    if img_vec is None or txt_vec is None:
        skipped += 1
        continue

    X.append(np.concatenate([img_vec, txt_vec]))
    y.append(target)
    image_names.append(img_name)

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Пропущено строк:", skipped)

После объединения визуальных и текстовых эмбеддингов сформирована итоговая обучающая выборка размером 4175 объектов с общей размерностью признакового пространства 2816.
1521 объектов были исключены, так как для них отсутствовал либо вектор изображения, либо вектор текста.

**Обучение модели предсказания соответствия**

In [ ]:
df = pd.DataFrame(X)
df['target'] = y
df['image_name'] = image_names



In [ ]:
df = pd.DataFrame(X)
df['target'] = y
df['image_name'] = image_names

gss = GroupShuffleSplit(n_splits=1, train_size=0.7, random_state=42)

train_idx, val_idx = next(
    gss.split(
        X=df.drop(columns=['target']),
        y=df['target'],
        groups=df['image_name']
    )
)

train_df = df.loc[train_idx].reset_index(drop=True)
val_df   = df.loc[val_idx].reset_index(drop=True)

X_train = train_df.drop(columns=['target', 'image_name']).values
y_train = train_df['target'].values

X_val = val_df.drop(columns=['target', 'image_name']).values
y_val = val_df['target'].values

print("Train:", X_train.shape)
print("Val  :", X_val.shape)


In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

In [ ]:
ridge = Ridge(random_state=42)

param_grid = {
    "alpha": [0.01, 0.1, 1, 10, 100]
}

grid = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=5,
    n_jobs=-1
)


grid.fit(X_train_scaled, y_train)

best_ridge = grid.best_estimator_

y_val_pred_ridge = best_ridge.predict(X_val_scaled)

print("Best alpha:", grid.best_params_["alpha"])
print("Ridge Regression (validation):")
print("MSE:", mean_squared_error(y_val, y_val_pred_ridge))
print("MAE:", mean_absolute_error(y_val, y_val_pred_ridge))
print("R2 :", r2_score(y_val, y_val_pred_ridge))

In [ ]:
nn_model = Sequential([
    Dense(1024, input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(256, activation='relu'),
    Dropout(0.2),

    Dense(1)
])

nn_model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='mse',
    metrics=['mae']
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = nn_model.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=2
)

In [ ]:
y_val_pred_nn = nn_model.predict(X_val_scaled).ravel()

print("Neural Network (validation):")
print("MSE:", mean_squared_error(y_val, y_val_pred_nn))
print("MAE:", mean_absolute_error(y_val, y_val_pred_nn))
print("R2 :", r2_score(y_val, y_val_pred_nn))


Были обучены и сравнены две модели регрессии: линейная модель с L2-регуляризацией (Ridge Regression) и полносвязная нейронная сеть.

Модель Ridge Regression была использована в качестве базовой линейной модели. По результатам валидации модель показала следующие значения метрик:

MSE = 0.082

MAE = 0.216

R² = −0.28



Отрицательное значение коэффициента детерминации указывает на то, что линейная модель не способна адекватно описать зависимость между эмбеддингами изображений и текстов и целевой экспертной оценкой. Фактически модель работает хуже, чем предсказание среднего значения.

Полносвязная нейронная сеть была обучена для моделирования нелинейных зависимостей между признаками. Использование нескольких скрытых слоёв, нормализации и регуляризации позволило модели учитывать сложные взаимодействия между визуальными и текстовыми представлениями.

По результатам валидации нейронная сеть показала лучшие результаты:

MSE = 0.049

MAE = 0.152

R² = 0.22

По сравнению с линейной моделью наблюдается:
снижение MSE, уменьшение MAE.

Полносвязная нейронная сеть  превосходит линейную модель и является предпочтительным решением для данной задачи.

**Тестирование модели и демонстрация ее работы**

In [ ]:

def contains_blocked_words(text: str) -> bool:
    if not isinstance(text, str):
        return False

    text = text.lower()
    tokens = re.findall(r"\b\w+\b", text)

    return any(token in BLOCK_LEMMAS for token in tokens)

In [ ]:
def text_to_vector(text, max_length=64):
    if not isinstance(text, str):
        return None

    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = bert_model(**inputs)

    vector = outputs.last_hidden_state.mean(dim=1)
    return vector.squeeze().cpu().numpy()


In [ ]:
def find_top_images(
    text: str,
    model,
    image_vectors: dict,
    top_n: int = 10
):

    if contains_blocked_words(text):
        return [("⚠️ Disclaimer: request may involve sensitive content.", None)]


    text_vec = text_to_vector(text)
    if text_vec is None:
        return []

    scores = []

    for img_name, img_vec in image_vectors.items():
        x = np.concatenate([img_vec, text_vec]).reshape(1, -1)
        score = model.predict(x, verbose=0)[0][0]
        scores.append((img_name, float(score)))


    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[:top_n]


In [ ]:
def show_images_grid(image_names, images_dir=IMAGES_DIR, cols=5):
    rows = (len(image_names) + cols - 1) // cols
    plt.figure(figsize=(4 * cols, 4 * rows))

    for i, img_name in enumerate(image_names):
        img_path = os.path.join(images_dir, img_name)
        if os.path.exists(img_path):
            img = PILImage.open(img_path).convert("RGB")
            plt.subplot(rows, cols, i + 1)
            plt.imshow(img)
            plt.axis("off")
            plt.title(img_name, fontsize=8)

    plt.tight_layout()
    plt.show()


In [ ]:
queries = [
    "child playing in sandbox",
    "kids painting on paper",
    "happy child with toy",
    "dog running in park",
    "people walking in the city"
]

top_n = 10

for q in queries:
    print(f"Query: {q}")

    results = find_top_images(
        text=q,
        model=nn_model,
        image_vectors=image_vectors,
        top_n=top_n
    )

    image_names = []

    for img_name, score in results:
        if img_name is None or "Disclaimer" in str(img_name):
            print(img_name)
        else:
            print(f"Image: {img_name}, Score: {score:.3f}")
            image_names.append(img_name)

    if image_names:
        show_images_grid(image_names, cols=5)

    print("-" * 60)


Видно, что модель склонна выдавать ограниченный набор изображений. Модель способна выявлять общее соответствие между текстовыми описаниями и изображениями и корректно выделяет часть релевантных изображений в верхних позициях выдачи. При этом наблюдается слабая дифференциация результатов для разных запросов: наборы топ-изображений во многом пересекаются, а различия в оценках невелики. Это свидетельствует о том, что модель улавливает лишь грубую семантическую близость и недостаточно чувствительна к деталям текстового запроса.

**Вывод**

- Подготовка данных
Был сформирован единый датасет, содержащий:

векторные представления изображений,

текстовые описания,

целевую метрику сходства «картинка–текст».
Для корректной оценки качества данные были разделены на обучающую, валидационную и тестовую выборки с учетом группировки по изображениям, что позволило избежать утечки данных.

- Векторизация текста
Для преобразования текстовых запросов в числовые признаки использовалась модель BERT. Текст переводился в эмбеддинг фиксированной размерности путем усреднения скрытых состояний, что позволило получить семантически осмысленное представление запросов.

- Использование визуальных признаков
Для изображений применялись заранее полученные векторные представления, отражающие визуальное содержание картинок. Это позволило работать с изображениями в числовом виде без обучения сверточной сети с нуля.

- Построение моделей
Были реализованы и сравнены несколько подходов:

линейная регрессия и ridge-регрессия как базовые модели,

полносвязная нейронная сеть, способная учитывать нелинейные зависимости между текстовыми и визуальными признаками.

- Оценка качества моделей
Качество моделей оценивалось с помощью метрик MSE, MAE и R² на валидационной выборке. Нейронная сеть показала лучшие результаты по всем ключевым метрикам по сравнению с линейными моделями.

Лучшие результаты показала полносвязная нейронная сеть, обученная на  векторах изображения и текста. По сравнению с линейной и ridge-регрессией, нейросетевая модель продемонстрировала более низкие значения MSE и MAE, а также положительное значение коэффициента детерминации, что указывает на способность модели улавливать нелинейные зависимости между визуальными и текстовыми признаками.

Во время поиска изображений по текстовому запросу модель допускает ряд  ошибок. Во-первых, она часто выдает схожие наборы изображений для разных, но семантически близких запросов, что говорит о слабой чувствительности к деталям текста.

Во-вторых, в топ-результатах нередко присутствуют изображения, соответствующие лишь общей сцене или контексту, но не ключевым объектам запроса. Кроме того, оценки сходства между изображениями часто отличаются незначительно.

В целом проект по созданию сервиса поиска фотографий по текстовому описанию  осуществим: даже базовая архитектура демонстрирует способность находить релевантные изображения и корректно ранжировать часть результатов. Однако для использования в реальных условиях требуется дальнейшее улучшение качества.